## Орг. вопросы

### Настройка среды

Кода написан для Python 3.6.0.
Скорее всего, с другими версиями Python проблем не возникнет, но может понадобиться немного адаптировать код.

Проще всего установить Anaconda: https://www.continuum.io/downloads <br/>
Там уже есть почти все нужные нам библиотеки.

Можно все установить и вручную. В этом случае рекомендую посмотреть на pyenv. Он позволяет держать на компьютере несколько виртуальных сред с разными версиями python и набором библиотек.<br/>
Здесь есть неплохой мануал по использованию: http://www.chriskrycho.com/2015/a-modern-python-development-toolchain.html <br/>
Мануал для Mac, но если использовать apt-get вместо brew, все должно заработать и на linux.

Вот список библиотек, которые понадобятся (если у вас Anaconda, все кроме pymorphy уже должно стоять):
```
numpy scipy pandas jupyter matplotlib scikit-learn nltk pymorphy2[fast] pymorphy2-dicts-ru
```

## Извлечение признаков из текста

На этом семинаре мы рассмотрим подходы и доступные библиотеки, позволяющие собирать признаки из текстов.

### NLTK

Распространенная библиотека для обработки текстов. Содержит алгоритмы, корпусы, обученные модели. <br/>
http://www.nltk.org <br/>
Есть книга по обработке текста с использованием NLTK: http://www.nltk.org/book/

In [1]:
import nltk

Вместе с nltk идет набор данных, который не скачивается по-умолчанию. <br/>
Для выполнения кода, приведенного ниже может понадобиться скачать дополнительные файлы. Для этого запустите код:

In [ ]:
nltk.download() # Откроется обычное GUI-окно. Закройте его, чтобы продолжить выполнение ноутбука.

#### Токенизация

Разбиение текста на токены (слова, пунктуация, числа, url, телефоны, ...)

http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize

text = "Mr. Smith! Glad to see you. $3.55"

print(sent_tokenize(text))
print(word_tokenize(text))

['Mr. Smith!', 'Glad to see you.', '$3.55']
['Mr.', 'Smith', '!', 'Glad', 'to', 'see', 'you', '.', '$', '3.55']


Нужно следить, чтобы токенизация правильно работала на ваших текстах. <br/>Например, на текстах из twitter стандартный токенизатор может работать плохо:

In [3]:
text = "hello @username!!!!! :) http://alink.com/page#part #hashtag"
print(word_tokenize(text))

['hello', '@', 'username', '!', '!', '!', '!', '!', ':', ')', 'http', ':', '//alink.com/page', '#', 'part', '#', 'hashtag']


In [4]:
from nltk.tokenize.casual import TweetTokenizer
tokenizer = TweetTokenizer(reduce_len=True)
tokenizer.tokenize(text)

['hello',
 '@username',
 '!',
 '!',
 '!',
 ':)',
 'http://alink.com/page#part',
 '#hashtag']

Иногда помогает объединить несколько токенов в один. Например, устойчивые выражения или конструкции вида $100, 10x15.

In [5]:
from nltk.tokenize import MWETokenizer
tokenizer = MWETokenizer([('a', 'little'), ('a', 'little', 'bit'), ('a', 'lot')])
tokenizer.add_mwe(('in', 'spite', 'of'))
tokenizer.tokenize(word_tokenize('In a little or a little bit or a lot in spite of'))

['In', 'a_little', 'or', 'a_little_bit', 'or', 'a_lot', 'in_spite_of']

Даже такая задача, как разбиение на токены - не тривиальная. <br/>
Еще сильней она усложняется, если в тексте встречаются опечатки, или он получен с помощью OCR.

В некоторых случаях правильное деление на токены можно выполнить только с учетом знаний о морфологии, синтаксисе. В этом случае можно порождать разные гипотезы разбиения на токены (граф линейного деления), и отбирать их на следующем этапе анализа.

#### Стоп-слова

Списки стоп-слов используются для того, чтобы игнорировать слова, которые не помогают в решении задачи.

Это могут быть конструкции, которые не несут самостоятельного смысла в предложении. Для классификации и кластеризации могут быть бесполезны высокочастотные слова, которые есть почти в каждом тексте.

Фильтрация стоп-слов иногда помогает повысить качество

In [6]:
from nltk.corpus import stopwords
for w in stopwords.words("russian"):
    print(w)

и
в
во
не
что
он
на
я
с
со
как
а
то
все
она
так
его
но
да
ты
к
у
же
вы
за
бы
по
только
ее
мне
было
вот
от
меня
еще
нет
о
из
ему
теперь
когда
даже
ну
вдруг
ли
если
уже
или
ни
быть
был
него
до
вас
нибудь
опять
уж
вам
ведь
там
потом
себя
ничего
ей
может
они
тут
где
есть
надо
ней
для
мы
тебя
их
чем
была
сам
чтоб
без
будто
чего
раз
тоже
себе
под
будет
ж
тогда
кто
этот
того
потому
этого
какой
совсем
ним
здесь
этом
один
почти
мой
тем
чтобы
нее
сейчас
были
куда
зачем
всех
никогда
можно
при
наконец
два
об
другой
хоть
после
над
больше
тот
через
эти
нас
про
всего
них
какая
много
разве
три
эту
моя
впрочем
хорошо
свою
этой
перед
иногда
лучше
чуть
том
нельзя
такой
им
более
всегда
конечно
всю
между


#### Стэмминг

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
print(ps.stem("cats"))
print(ps.stem("running"))
print(ps.stem("meeting"))

cat
run
meet


#### Лемматизация

In [34]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("running"))
print(lemmatizer.lemmatize("meeting"))
print(lemmatizer.lemmatize("better"))
print(lemmatizer.lemmatize("better", pos="a"))

cat
running
meeting
better
good


#### POS-тэггинг

Part of Speech тэггинг позволяет разметить токены тегами частей речи.

In [9]:
text = nltk.word_tokenize("She saw the man with a telescope")
text

['She', 'saw', 'the', 'man', 'with', 'a', 'telescope']

In [10]:
nltk.pos_tag(text)

[('She', 'PRP'),
 ('saw', 'VBD'),
 ('the', 'DT'),
 ('man', 'NN'),
 ('with', 'IN'),
 ('a', 'DT'),
 ('telescope', 'NN')]

In [12]:
# Получить список доступных тэгов можно с помощью команды:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

Часть речи может зависеть от контекста, в котором употреблено слово:

In [13]:
print(nltk.pos_tag(nltk.word_tokenize("A fly to London")))
print(nltk.pos_tag(nltk.word_tokenize("I fly to London")))
print(nltk.pos_tag(nltk.word_tokenize("I opened a window")))

[('A', 'DT'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NNP')]
[('I', 'PRP'), ('fly', 'VBP'), ('to', 'TO'), ('London', 'NNP')]
[('I', 'PRP'), ('opened', 'VBD'), ('a', 'DT'), ('window', 'NN')]


In [14]:
print(nltk.pos_tag(nltk.word_tokenize("I blablablowed the window")))

[('I', 'PRP'), ('blablablowed', 'VBD'), ('the', 'DT'), ('window', 'NN')]


Посмотрим, что если назначать часть речи каждому слову наивно - наиболее частотную часть речи:

In [21]:
# Воспользуемся brown корпусом, входящим в состав NLTK.
# Он размечен тегами частей речи. Посмотрим на несколько первых токенов:
from nltk.corpus import brown
print(brown.tagged_words(categories='news')[:20])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS')]


In [22]:
# Теперь обучим на корпусе UnigramTagger - теггер, который выбирает для каждого слова ту часть речи, 
# с которой оно чаще всего встречалось в обучающем корпусе.
from nltk.tag import UnigramTagger
tagger = UnigramTagger(brown.tagged_sents(categories='news')[:3000]) # Возьмем первые 3000 токенов
print(tagger.tag(nltk.word_tokenize("I fly to London")))
print(tagger.tag(nltk.word_tokenize("A fly to London")))

[('I', 'PPSS'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NP')]
[('A', 'AT'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NP')]


In [24]:
# Проверим качество на отложенном фрагменте brown-корпуса.
tagger.evaluate(brown.tagged_sents(categories='news')[3000:])

0.7878104465976167

Кажется, что цифра 0.78 достаточно высокая для такого наивного подхода.<br/>
Но если задуматься, то это значит, что часть речи каждого пятого слова была определена.

#### Морфологический анализ - Pymorphy

Для русского и украинского языка вы можете использовать PyMorphy<br/>
https://pymorphy2.readthedocs.io/en/latest/user/guide.html

In [28]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
# Посмотрим варианты морфологического разбора слова "стали":
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [29]:
# Получим наиболее вероятный тег и часть речи:
tag = morph.parse('стали')[0].tag
print(tag)
print(tag.POS)

VERB,perf,intr plur,past,indc
VERB


In [30]:
# Приведение к нормальной форме:
morph.parse('думающему')[0].normal_form

'думать'

Другие инструменты для работы с русским языком (POS-тэггинг, синтаксический анализ) доступны на странице<br/>
http://corpus.leeds.ac.uk/mocky/

### Регулярные выражения

С помощью регулярных выражений можно осуществлять предобработку текстов, строить признаки-шаблоны на слова и подстроки текста и многое другое.<br/>
Документация для python:<br/>
https://docs.python.org/3.6/howto/regex.html <br/>
https://docs.python.org/3.6/library/re.html#module-re

Хочется обратить внимание на два полезных момента.<br/>
1) re.X позволяет использовать пробельные символы и делать комментарии в ваших регулярных выражениях. Это очень удобно!

In [31]:
import re
a = re.compile(r"""\d +  # the integral part
                   \.    # the decimal point
                   \d *  # some fractional digits""", re.X)
b = re.compile(r"\d+\.\d*")
a.search("asdf 1.1232")

<_sre.SRE_Match object; span=(5, 11), match='1.1232'>

2) Существуют не жадные версии операторов

In [35]:
# Не жадные версии операторов: +? *? ??
c = re.compile(r"\d+\.\d*?")
c.search("asdf 1.1232")

<_sre.SRE_Match object; span=(5, 7), match='1.'>